In [10]:
import os
import sys
import numpy as np
import time

np.set_printoptions(suppress=True, precision=4)
    
def set_lowVal_zero(X):
    low_values_indices = abs(X) < 9e-15   # where values are low
    X[low_values_indices] = 0             # all low values set to 0
    return X

def Householder(x, i):
    alpha = -np.sign(x[i]) * np.linalg.norm(x)
    e = np.zeros(len(x)); e[i] = 1.0
    
    v = (x - alpha * e)
    w = v / np.linalg.norm(v)
    P = np.eye(len(x)) - 2 * np.outer(w, w.T)
    
    return P

st = time.time()
A = np.array([[4, 3, 0, 2, 5], [2, 1, 2, 1, 6], [4, 4, 0, 3, 0], [5, 6, 1, 3, 7],[6,2,4,8,3]]);
def Golub_Kahan(X):
    col = X.shape[1]
    row = X.shape[0]
    J = X.copy()
    for i in range(col - 2):
        # column
        h = np.zeros(len(J[:, i]))
        h[i:] = J[i:, i]
        P = Householder(h, i)
        J = set_lowVal_zero(P @ J)
        #print(J, '\n')

        # row
        h = np.zeros(len(J[i, :]))
        h[i+1:] = J[i, i+1:] 
        Q = Householder(h, i+1)
        J = set_lowVal_zero(J @ Q)
        B = np.array([[3.5309,5.3093,0],[0,-3.2272,3.2113],[0,0,3.224]])
        b_transpose = np.transpose(B)
        b = b_transpose.dot(B)
        C = [[]]
        C = np.array(b[1:,1:])
        lambda1,lambda2 = np.linalg.eigvals(C)
        #print(lambda1,lambda2);
        mu1 = abs(C[1][1]-lambda1)
        mu2 = abs(C[1][1]-lambda2);B=J;
        mu=0
        if(mu1>mu2 and mu1>0):
            mu = lambda2
        else:
            mu = lambda1
        p=int(-3.2272)
        q = int(5.3093)
        n = 2

        for k in range(p+1,n-q-1):
            #Right Rotation matrix
            alpha = B[k][k]**2 - mu
            beta = B[k][k+1]
            x = (alpha*2 + beta**2)**0.5
            c_r = x/(alpha+((beta**2)/alpha))
            s_r = (-1*beta*c)/alpha
            R = np.array([[1,0,0],[0,c_r,-s_r],[0,s_r,c_r]])
            B = B.dot(R)
            print("B is:",B)
            #left Rotation Matrix
            alpha = B[k][k]
            beta = B[k+1][k]
            c_l = x/(alpha-((beta**2)/alpha))
            s_l = (c_l*beta)/alpha
            L = np.array([[1,0,0],[0,c_r,s_r],[0,-s_r,c_r]])
            B = B.dot(L)
            E = np.linalg.norm(A);B=J;
            if(k<=n-q-1):
                alpha = B[k][k+1]
                beta = B[k][k+2]
    return B
J = Golub_Kahan(A)
#print(lambda1,lambda2);
sp = time.time()
print("The time for Golub-Kahan is:\n",sp-st)
print(J)


The time for Golub-Kahan is:
 0.11070513725280762
[[ -9.8489  14.6164   0.       0.       0.    ]
 [  0.       3.5309   5.3093   0.       0.    ]
 [  0.       0.      -3.2272   3.2113   0.    ]
 [  0.       0.       0.       3.224   -0.5598]
 [  0.       0.       0.      -2.3999   1.2292]]
